# Hồi quy đa biến Multiple Regression (Diễn giải)

Trong notebook này, chúng ta sẽ sử dụng dữ liệu doanh số bán nhà ở Quận King để dự đoán giá nhà sử dụng hồi quy tuyến tính đa biến. Chúng ta sẽ:
* Thực hiện một số thiết kế đặc trưng bằng các hàm DataFrame nội bộ.
* Sử dụng các hàm sklearn có sẵn để tính hồi quy và truy cập các tham số của nó (hệ số).
* Viết hàm tính RSS với các trọng số hồi quy, yếu tố dự báo và đầu ra đã cho.
* Xem các hệ số và diễn giải ý nghĩa của chúng.
* Đánh giá mô hình đa biến qua RSS.

## Import thư viện

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sklearn, pandas
import numpy as np

## Load dữ liệu bán nhà

Tập dữ liệu từ doanh số bán nhà quận King, Seatle, WA.

In [ ]:
full_data = pandas.read_csv("/content/drive/MyDrive/Colab_Notebooks/Bai tap lap/Mon_hoc_2/kc_house_data.csv", index_col=0)

In [ ]:
print(full_data.describe())
print(full_data.info())

              price      bedrooms     bathrooms   sqft_living      sqft_lot  \
count  2.161300e+04  21613.000000  21613.000000  21613.000000  2.161300e+04   
mean   5.400881e+05      3.370842      2.114757   2079.899736  1.510697e+04   
std    3.671272e+05      0.930062      0.770163    918.440897  4.142051e+04   
min    7.500000e+04      0.000000      0.000000    290.000000  5.200000e+02   
25%    3.219500e+05      3.000000      1.750000   1427.000000  5.040000e+03   
50%    4.500000e+05      3.000000      2.250000   1910.000000  7.618000e+03   
75%    6.450000e+05      4.000000      2.500000   2550.000000  1.068800e+04   
max    7.700000e+06     33.000000      8.000000  13540.000000  1.651359e+06   

             floors    waterfront          view     condition         grade  \
count  21613.000000  21613.000000  21613.000000  21613.000000  21613.000000   
mean       1.494309      0.007542      0.234303      3.409430      7.656873   
std        0.539989      0.086517      0.766318    

## Chia dữ liệu thành tập huấn luyện và kiểm tra.
Chúng ta sử dụng seed=0 để mọi người chạy notebook này có đều được kết quả tương tự. Thực tế, các bạn có thể thiết lập seed ngẫu nhiên để đảm bảo tính nhất quán.

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(full_data, train_size=0.8, test_size=0.2, random_state=0)
train_data = train_data.copy()
test_data = test_data.copy()

# Tìm hiểu mô hình hồi quy đa biến

Chúng ta có thể sử dụng code sau để tìm hiểu mô hình hồi quy đa biến dự đoán 'price' dựa trên các đặc trưng sau: example_features = ['sqft_living', 'bedrooms', 'bathrooms'] trong dữ liệu huấn luyện với code:

In [ ]:
def extract_features(data, features_title):
    # nhận các đặc trưng từ DataFrame
    features = [data[title].values for title in features_title]
    # xếp chồng chúng thành 2d [examples, properties]
    return np.stack(features, axis=-1)

In [ ]:
from sklearn.linear_model import LinearRegression

example_features_title = ['sqft_living', 'bedrooms', 'bathrooms']
# trích xuất đặc trưng thành các mảng numpy và xếp chồng chúng mỗi example
example_features = extract_features(train_data, example_features_title)
example_labels = train_data['price']
example_model = LinearRegression().fit(example_features, example_labels)

Chúng ta đã khớp mô hình, giờ có thể trích xuất các trọng số mô hình (hệ số) từ mô hình như sau:

In [ ]:
example_weight_summary = example_model.coef_
print(example_weight_summary)

[   313.17055038 -56754.66651422   6887.71910816]


In [ ]:
print(example_features[:5])
print(test_data.iloc[:5])

[[1.57e+03 3.00e+00 1.00e+00]
 [1.78e+03 3.00e+00 2.50e+00]
 [1.09e+03 3.00e+00 1.50e+00]
 [2.21e+03 4.00e+00 2.50e+00]
 [1.80e+03 3.00e+00 2.50e+00]]
                       date      price  bedrooms  bathrooms  sqft_living  \
id                                                                         
1453602313  20141029T000000   297000.0         2       1.50         1430   
2225059214  20140808T000000  1578000.0         4       3.25         4670   
2768000270  20140625T000000   562100.0         2       0.75         1440   
6819100040  20140624T000000   631500.0         2       1.00         1130   
4027700666  20150426T000000   780000.0         4       2.50         3180   

            sqft_lot  floors  waterfront  view  condition  grade  sqft_above  \
id                                                                             
1453602313      1650     3.0           0     0          3      7        1430   
2225059214     51836     2.0           0     0          4     12        4670

## Đưa ra dự đoán

Trong notebook về gradient descent, chúng ta sử dụng numpy để thực hiện hồi quy. Trong notebook này, chúng ta sẽ sử dụng các hàm giao diện của mô hình sklearn hiện có để phân tích hồi quy đa biến.

Khi mô hình đã xây, chúng ta có thể sử dụng hàm `predict` để tìm các giá trị dự đoán cho dữ liệu mà chúng ta truyền vào. Ví dụ: sử dụng example_model trên:

In [ ]:
example_predictions = example_model.predict(example_features)
print(example_predictions[0]) # should be 395813.499

395813.4988028938


## Tính RSS

Bây giờ chúng ta có thể đưa ra các dự đoán cho mô hình, hãy viết một hàm tính RSS của mô hình. Hoàn thành hàm dưới đây để tính RSS với mô hình, dữ liệu và đầu ra đã biết.

In [ ]:
def get_residual_sum_of_squares(model, data, outcome):
    # Trước tiên lấy các dự đoán
    predict_out = model.predict(data)
    # Sau đó tính các phần dư/lỗi
    residu = predict_out - outcome
    # Bình phương lên và cộng tổng
    RSS = residu@residu.T
    return(RSS)

In [ ]:
#==================tesst
example_test_features = extract_features(test_data, example_features_title)
example_test_labels = test_data['price']
predict_tess =  example_model.predict(example_test_features)
print(type(predict_tess))
print(predict_tess[:5])
print(predict_tess.shape)

<class 'numpy.ndarray'>
[ 412168.14781776 1325384.9064653   410134.06399046  314773.12314917
  853594.99706545]
(4323,)


In [ ]:
print(example_test_labels.shape)

(4323,)


In [ ]:
x = np.array([5,6,7,8])
print(x@x.T)
y = np.array([5,6,7,8])
print(x-y)

174
[0 0 0 0]


Kiểm tra hàm bằng cách tính RSS trong dữ liệu KIỂM TRA cho mô hình mẫu:

In [ ]:
example_test_features = extract_features(test_data, example_features_title)
example_test_labels = test_data['price']
rss_example_test = get_residual_sum_of_squares(example_model, example_test_features, example_test_labels)
print(rss_example_test) # should be ~ 2.5921e+14

259213572106085.4


# Tạo một số đặc trưng mới

Chúng ta thường nghĩ hồi quy đa biến gồm nhiều đặc trưng khác nhau (ví dụ: số phòng ngủ, diện tích và số phòng tắm), nhưng chúng ta cũng có thể xem xét việc biến đổi các đối đặc trưng hiện có, ví dụ: log của squarefeet hoặc thậm chí các đặc trưng "tương tác" như tích của số phòng ngủ và số phòng tắm.

Giờ chúng ta sẽ sử dụng hàm logarit mặc định của python để tạo đặc trưng mới. Chúng ta cần import nó từ thư viện math.

In [ ]:
from math import log

Tiếp theo, chúng ta sẽ tạo 4 đặc trưng mới sau làm cột trong cả dữ liệu HUẤN LUYỆN và KIỂM TRA:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long

Ví dụ như sau:

In [ ]:
print(train_data.shape)
print(test_data.shape)

(17290, 24)
(4323, 24)


In [ ]:
import math

In [ ]:
train_data['bedrooms_squared'] = train_data['bedrooms'].map(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].map(lambda x: x**2)

In [ ]:
# tạo 3 đặc trưng còn lại trong cả dữ liệu HUẤN LUYỆN và KIỂM TRA
train_data['bed_bath_rooms'] = train_data['bedrooms'].mul(train_data['bathrooms'])
test_data['bed_bath_rooms'] = test_data['bedrooms'].mul(test_data['bathrooms'])

train_data['log_sqft_living'] = train_data['sqft_living'].map(lambda x : math.log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].map(lambda x : math.log(x))

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

In [ ]:
print(train_data.iloc[:3])

                       date     price  bedrooms  bathrooms  sqft_living  \
id                                                                        
5100402668  20150218T000000  495000.0         3        1.0         1570   
7856560480  20140808T000000  635000.0         3        2.5         1780   
2872900010  20150414T000000  382500.0         3        1.5         1090   

            sqft_lot  floors  waterfront  view  condition  ...  yr_renovated  \
id                                                         ...                 
5100402668      5510     1.0           0     0          4  ...             0   
7856560480     11000     1.0           0     0          4  ...             0   
2872900010      9862     1.0           0     0          3  ...             0   

            zipcode      lat     long  sqft_living15  sqft_lot15  \
id                                                                 
5100402668    98115  47.6942 -122.319           1770        6380   
7856560480    98006

* bedrooms*bedrooms sẽ tăng phân tách giữa ít phòng ngủ (chẳng hạn: 1) và nhiều phòng ngủ (chẳng hạn: 4). Do đó, đặc trưng này phần lớn sẽ ảnh hưởng với nhiều phòng ngủ.
* bedrooms*bathrooms cho đặc trưng "tương tác", nó sẽ lớn khi cả hai đều lớn.
* log(sqft_living) khiến các giá trị lớn hơn gần nhau hơn và lan ra các giá trị nhỏ.
* lat + long hoàn toàn không có ý nghĩa nhưng chúng ta vẫn thực hiện (bạn sẽ biết lý do sau).

**Quiz: Giá trị trung bình cộng của 4 đặc trưng mới trong dữ liệu KIỂM TRA là bao nhiêu? (làm tròn tới 2 chữ số thập phân)**

In [ ]:
# pandas.Series có hàm gọi chính xác như vậy.
print((len(train_data) + len(test_data)))
print((train_data['bedrooms_squared'].sum()+ test_data['bedrooms_squared'].sum())/((len(train_data) + len(test_data))))
print((train_data['bed_bath_rooms'].sum()+ test_data['bed_bath_rooms'].sum())/((len(train_data) + len(test_data))))
print((train_data['log_sqft_living'].sum()+ test_data['log_sqft_living'].sum())/((len(train_data) + len(test_data))))
print((train_data['lat_plus_long'].sum()+ test_data['lat_plus_long'].sum())/((len(train_data) + len(test_data))))

21613
12.22754823485865
7.498022023781983
7.550334713314153
-74.6538438856244


## Tìm hiểu mô hình đa biến

Bây giờ chúng ta sẽ tìm hiểu trọng số của ba mô hình (lồng nhau) dự đoán giá nhà. Mô hình đầu tiên sẽ ít đặc trưng nhất, mô hình thứ hai sẽ thêm một đặc trưng và mô hình thứ ba sẽ thêm vài đặc trưng khác:
* Mô hình 1: sqft_living, # bedrooms, # bathrooms, lat & long
* Mô hình 2: thêm bedrooms\*bathrooms
* Mô hình 3: thêm log_sqft, bedrooms_squared, và lat_plus_long

In [ ]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Bạn đã có các đặc trưng, trọng số cho 3 mô hình khác nhau dự đoán target = 'price' sử dụng LinearRegression của sklearn và thấy giá trị trọng số/hệ số:

*Lưu ý: mô hình hồi quy trong khóa này thường có `coef_`*

In [ ]:
# Tìm hiểu 3 mô hình. Trích xuất đặc trưng và khớp mô hình với các đặc trưng tương ứng.
# Nếu không nhớ, hãy xem example_model bên trên
example_1_features = extract_features(train_data, model_1_features)
example_2_features = extract_features(train_data, model_2_features)
example_3_features = extract_features(train_data, model_3_features)
example_1_model =  LinearRegression().fit(example_1_features, example_labels)
example_2_model =  LinearRegression().fit(example_2_features, example_labels)
example_3_model =  LinearRegression().fit(example_3_features, example_labels)

In [ ]:
# Kiểm tra/trích xuất từng hệ số của mô hình. Nếu nghi ngỡ, hãy tham khảo tài liệu của sklearn.
print(example_1_model.coef_)
print(example_2_model.coef_)
print(example_3_model.coef_)

[ 3.12942010e+02 -5.30962691e+04  1.47770428e+04  6.53983343e+05
 -3.25707336e+05]
[ 3.06819573e+02 -1.04604718e+05 -7.01815289e+04  6.50590952e+05
 -3.09965751e+05  2.49441497e+04]
[ 5.37808085e+02  2.78047828e+03  1.01363764e+05  1.99440683e+12
  1.99440589e+12 -1.81822547e+04  7.24579903e+02 -5.71030022e+05
 -1.99440618e+12]


**Quiz: Dấu (dương hoặc âm) của hệ số/trọng số cho 'bathrooms' trong mô hình 1 là gì?**

**Quiz: Dấu (dương hoặc âm) của hệ số/trọng số cho 'bathrooms' trong mô hình 2 là gì?**

Hãy nghĩ xem điều này có ý nghĩa gì.

In [ ]:
# Trả lời: hệ số dương trong mô hình 1 cho biết mức tăng của giá là 14.777 khi tăng 1 bathrooms, khi đó các features khác bathrooms fixed
# Trả lời: hệ số âm trong mô hình 1 cho biết mức giảm của giá là 70181 khi tăng 1 bathrooms, khi đó các features khác bathrooms fixed
# ở các ngữ cảnh (context) khác nhau ta có cách diễn giải (interpreting) cho cùng 1 feature là khác nhau

## So sánh các mô hình đa biến

Chúng ta đã nghiên cứu 3 mô hình và trích xuất trong số mô hình mà chúng ta muốn đánh giá là tốt nhất.

Trước tiên sử dụng các hàm trước đó để tính RSS trong dữ liệu HUẤN LUYỆN cho từng mô hình.

In [ ]:
# Tính RSS trong dữ liệu HUẤN LUYỆN cho từng mô hình và hiển thị các giá trị.
# Xem lab trước nếu quên cách làm.
rss_example1_train = get_residual_sum_of_squares(example_1_model, example_1_features, example_labels)
rss_example2_train = get_residual_sum_of_squares(example_2_model, example_2_features, example_labels)
rss_example3_train = get_residual_sum_of_squares(example_3_model, example_3_features, example_labels)
print("RSS of 1 2 3, respectively is: \n{:.2f}\n{:.2f}\n{:.2f}".format(rss_example1_train,rss_example2_train,rss_example3_train))

RSS of 1 2 3, respectively is: 
979843597588329.75
970799199729577.25
913653643021650.75


**Quiz: Mô hình nào (1, 2 hay 3) có RSSS thấp nhất trong dữ liệu HUẤN LUYỆN?** Đây có phải điều chúng ta dự kiến?

** Mô hình 3: đúng dự kiến

Bây giờ hãy tính RSS trong dữ liệu KIỂM TRA cho từng mô hình.

In [ ]:
# Tính RSS trong dữ liệu KIỂM TRA cho từng mô hình và hiển thị các giá trị.
example_test_1_features = extract_features(test_data, model_1_features)
example_test_2_features = extract_features(test_data, model_2_features)
example_test_3_features = extract_features(test_data, model_3_features)
rss_example1_test = get_residual_sum_of_squares(example_1_model, example_test_1_features, example_test_labels)
rss_example2_test = get_residual_sum_of_squares(example_2_model, example_test_2_features, example_test_labels)
rss_example3_test = get_residual_sum_of_squares(example_3_model, example_test_3_features, example_test_labels)
print("RSS of 1 2 3, respectively is: \n{:.2f}\n{:.2f}\n{:.2f}".format(rss_example1_test,rss_example2_test,rss_example3_test))

RSS of 1 2 3, respectively is: 
213487129319104.06
210778544168942.56
203972160750041.06


**Quiz: Mô hình nào (1, 2 hay 3) có RSSS thấp nhất trong dữ liệu KIỂM TRA?** Đây có phải điều chúng ta dự kiến? Nghĩ về các đặc trưng đã thêm vào từng mô hình trước đó.

In [ ]:
# mô hình số 3, đúng dự kiến ứng với train